импорты

In [227]:
import pandas as pd
import numpy as np
import camelot
import os
import re
import json
import gc
import time
import shutil
import pathlib




## Тут мы написали функции препроцессинга

In [ ]:
def preprocessor(table: pd.DataFrame) -> dict[str, pd.DataFrame]:
    """
    Preprocess table
    """
    table_name = table.iloc[0]
    table_columns = table.iloc[1]
    table.columns = table_columns
    table.columns.name = None
    table = table.drop(index=[0, 1])
    table = table.reset_index(drop=True)
    return {str(table_name): pd.DataFrame(table)}

In [229]:
RE_DATE = re.compile(r"\b\d{1,2}[./-]\d{1,2}[./-]\d{2,4}\b")
RE_LONG_CODE = re.compile(r"\b(?:\d{1,4}\.){2,}\d{1,4}\b") 
RE_YEAR = re.compile(r"\b(19|20)\d{2}\b")
RE_TRAIL_DOTNUM = re.compile(r"(?<=\s)[\.\-]\d{1,3}\b")    
RE_PERCENT_TAIL = re.compile(r"\d+\s?%.*$")                
RE_MULTI_SPACE = re.compile(r"\s+")
RE_SP_BEFORE_PUNCT = re.compile(r"\s+([,.;:])")
RE_PUNCT_DUP = re.compile(r"[,.]{2,}")

TAILKEYS = r"(суха[яй]|мелк|помол|сечк|ток\d*|гранул|экструд|смесь|мешан|энпкх|энанпкх|энапкх|эн|энк)"
RE_TAIL_AFTER_COMMA = re.compile(rf",\s*.*?(?=($|\b))", re.IGNORECASE)
RE_TAILKEYS_AFTER_COMMA = re.compile(rf",\s*(?={TAILKEYS})[^\n]*", re.IGNORECASE)

In [230]:
def clean_ingredient(s: str) -> str:
    """
    Clear ingredients with regulat expressions
    """
    if not isinstance(s, str):
        s = "" if pd.isna(s) else str(s)

    s = s.replace("\n", " ").strip()

    s = re.sub(r"\bкомбиком\b", "комбикорм", s, flags=re.IGNORECASE)

    masks = {}
    # оставляем комбикормы номера
    def _mask(match):
        key = f"__MASKNO_{len(masks)}__"
        masks[key] = match.group(0)
        return key
    s = re.sub(r"№\s*\d+", _mask, s) 

    s = RE_DATE.sub("", s)
    s = RE_LONG_CODE.sub("", s)
    s = RE_YEAR.sub("", s)
    s = RE_TRAIL_DOTNUM.sub("", s)

    s = RE_PERCENT_TAIL.sub("", s)

    s = RE_TAILKEYS_AFTER_COMMA.sub("", s)

    s = s.replace("/", " ")

    s = RE_SP_BEFORE_PUNCT.sub(r"\1", s)
    s = RE_PUNCT_DUP.sub(lambda m: m.group(0)[0], s)
    s = RE_MULTI_SPACE.sub(" ", s).strip(" ,.;:")

    for key, val in masks.items():
        s = s.replace(key, val)

    s = s.rstrip(".,;: ").strip()

    return s

## Наши культуры и ингридиенты

In [222]:
with open('../data/cultures.json', 'r') as f:
    cultures = json.load(f)
cultures = pd.DataFrame.from_dict(cultures)

In [223]:
cultures = cultures.drop(columns=['code', 'name_en'])

In [226]:
cultures.head()

,name_ru,type
0,Сенаж,feed_form
1,Силос,feed_form
2,Сено,feed_form
3,Солома,feed_form
4,Концентраты,feed_form


## Тут мы начинаем парсить таблички с ингридиентами

In [ ]:
def main(list_ingredients):
    """
    Parsing tables
    """
    path = pathlib.Path('../data')
    dirs = [x for x in path.iterdir() if x.is_dir()]
        
    for d in dirs:
        files =  [x for x in d.iterdir() if x.suffix == '.pdf']
        for f in files:
            try:
                tables = camelot.read_pdf(
                    f,
                    pages='all',
                    flavor='lattice'
                )
                data = tables[0]
                data_preproc = preprocessor(data)
                data_preproc[str(f)]['Ингридиенты'] = data_preproc[str(f)]['Ингридиенты'].map(clean_ingredient)


                list_ingredients[f] = data_preproc
                
            except Exception as e:
                print(f"Ошибка при обработке файла {f}: {e}")
                time.sleep(0.5)  
                continue
    
    print("Обработка завершена успешно!")

#dic of pairs <name: table>
list_ingredients = {}
main(list_ingredients)

/Users/aleksandrandreev/EcoNiva-Cows/venv/lib/python3.13/site-packages/camelot/parsers/base.py:124: UserWarning: page-1 is image-based, camelot only works on text-based pages.
  warnings.warn(
/Users/aleksandrandreev/EcoNiva-Cows/venv/lib/python3.13/site-packages/camelot/parsers/base.py:124: UserWarning: page-2 is image-based, camelot only works on text-based pages.
  warnings.warn(


Ошибка при обработке файла ../data/ЭНА/Рацион Д1 ЖК Озерки 24.04.2024 ЭНАЮ.pdf: list index out of range
Обработка завершена успешно!


In [220]:
b = list_ingredients[pathlib.PosixPath('../data/ЭНА/Рацион МТФ Щучье 26.06.2025_ЭНАПБ.pdf')]

In [231]:
data = preprocessor(b.df)
data['Ингредиенты'] = data['Ингредиенты'].map(clean_ingredient)
data

,Ингредиенты,СВ %,ГП кг,СВ кг,% ГП,% СВ,₽/Тонна
0,Сено луговое,"94,70","0,74","0,70","1,81","3,18","3 000,00"
1,Сенаж,"34,70","10,66","3,70","26,16","16,82","4 858,00"
2,Силос,"39,80","17,59","7,00","43,16","31,82","3 582,00"
3,Шрот рапсовый,"91,80","2,83","2,60","6,95","11,82","19 600,00"
4,Кукуруза,"87,54","2,86","2,50","7,01","11,36","14 570,00"
5,комбикорм №13,"90,53","6,08","5,50","14,91","25,00","22 891,25"
